In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re


import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

In [3]:
def AggregateData(df, fase): 
    df['teller'] = pd.to_numeric(df['teller'])
    df = df.groupby('table').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
    df['Count'] = df.apply(lambda x: len(x.project), axis=1)
    df['Stage'] = fase
    df = df.sort_values('teller', ascending=False).reset_index()
    df = df.rename(columns={'table': 'Table', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_tabellen'}).sort_index()
    return df

In [4]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}},
            {"$group": { "_id": { 'project': "$project", 'table': "$table", 'teller': '$teller'}}},
            {'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$_id.teller"}}}]

df_brondata = AggregateData(pd.DataFrame(list(metaCollection.aggregate(grp_aggr))), 'Brondata')
df_brondata.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,AARDEWERK 1,"[DB34, DC97]",4618,2,Brondata
1,Aardewerk 1 backup,[DB34],4465,1,Brondata
2,VONDSTENLIJST,"[DC11, DB34, DC112, DC93, DC97]",2121,5,Brondata
3,Controle vondsten,[DB34],1466,1,Brondata
4,DIAOPGRAVING,"[DC93, DB34, DC97]",882,3,Brondata


In [5]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}}
           ,{"$group": { "_id": { 'project': "$project", 'table': "$table"}, 'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$teller"}}}]

df_staging = AggregateData(pd.concat([pd.DataFrame(list(stagingOud.aggregate(grp_aggr))), pd.DataFrame(list(stagingNieuw.aggregate(grp_aggr)))]), 'Staging')
df_staging.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,AARDEWERK 1,"[DB34, DC97]",4618,2,Staging
1,Aardewerk 1 backup,[DB34],4465,1,Staging
2,VONDSTENLIJST,"[DC11, DC112, DB34, DC97, DC93]",2121,5,Staging
3,Controle vondsten,[DB34],1466,1,Staging
4,DIAOPGRAVING,"[DC97, DC93, DB34]",882,3,Staging


In [13]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'table': "$brondata.table"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$teller"}}}]

df_singlestore = AggregateData(pd.DataFrame(list(analyseColClean.aggregate(grp_aggr))), 'SingleStore')
df_singlestore.head(50)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,magazijnlijst,[MAGAZIJN],17887,1,SingleStore
1,doosnr,[MAGAZIJN],2732,1,SingleStore
2,OPGRAVINGEN,[DELF-IT],840,1,SingleStore
3,SPOREN,"[DC97, DB34, DC112, DC93]",800,4,SingleStore
4,SPOOR,"[DC154, DC179]",479,2,SingleStore
5,GIS_SPOOR_STRUCTOBJ,[DC179],243,1,SingleStore
6,SPOOR_INHOUD,[DC179],208,1,SingleStore
7,REF_SPOORAARD,[DC112],96,1,SingleStore
8,VINDPLAATSEN,[DELF-IT],37,1,SingleStore
9,stellingen,[MAGAZIJN],22,1,SingleStore


In [7]:
lst_tables = ['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexTable = re.compile(r'\'table\': \'(.*?)\'') # regex to replace Object
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getTable(brondata):    
    #print(brondata)
    if not brondata or brondata != "":
        return regexTable.search(brondata).group(1)
    else: 
        return "" 
def getProject(brondata):    
    #print(brondata)
    if not brondata or brondata != "":
        return regexProject.search(brondata).group(1)
    else: 
        return "" 

df_tables_projects = pd.DataFrame()
    
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    for table in lst_tables:     
        df = pd.read_sql_query('SELECT brondata from "' + table + '"', connection)

        table_series = pd.Series(df.apply(lambda x: getTable(x['brondata']), axis=1))
        project_series = pd.Series(df.apply(lambda x: getProject(x['brondata']), axis=1))  
        frame = { 'table': table_series, 'project': project_series }
        df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(frame).groupby(['project', 'table']).size().reset_index(name='teller')])
    
df_doelsysteem = AggregateData(df_tables_projects, 'Doelsysteem')
df_doelsysteem.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,magazijnlijst,[MAGAZIJN],5634,1,Doelsysteem
1,AARDEWERK 1,"[DB34, DC97]",4618,2,Doelsysteem
2,VONDSTENLIJST,"[DB34, DC11, DC112, DC93, DC97]",2121,5,Doelsysteem
3,OPGRAVINGEN,[DELF-IT],840,1,Doelsysteem
4,ROMEINS AARDEWERK,[DB34],594,1,Doelsysteem


## Analyse van alle gegevens

Hieronder volgt een overzicht van alle gegevens die in de vier fases beschikbaar zijn. Zo kunnen fouten in de conversie worden opgespoord. 

In [8]:
def getOverview(OnWhat): 
    lst_countColumns = [OnWhat + '_brondata', OnWhat + '_staging', OnWhat + '_singlestore', OnWhat + '_doelsysteem']
    
    df = df_brondata[['Table', 'Projecten', OnWhat]].merge(df_staging[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_brondata", "_staging"))
    df = df.merge(df_singlestore[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_staging", "_singlestore"))
    df = df.merge(df_doelsysteem[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_singlestore", "_doelsysteem"))
    df = df.sort_values(OnWhat+ '_brondata', ascending=False) 
    df[lst_countColumns] = df[lst_countColumns].fillna(0)
    df[lst_countColumns] = df[lst_countColumns].astype(int, errors='ignore')
    df = df.style.bar(subset=lst_countColumns, color='#5fba7d', vmax=6000)
    
    return df

getOverview('Aantal_Records')

,Table,Projecten,Aantal_Records_brondata,Aantal_Records_staging,Aantal_Records_singlestore,Aantal_Records_doelsysteem
0,AARDEWERK 1,"['DB34', 'DC97']",4618,4618,0,4618
1,Aardewerk 1 backup,['DB34'],4465,4465,0,0
2,VONDSTENLIJST,"['DC11', 'DB34', 'DC112', 'DC93', 'DC97']",2121,2121,0,2121
3,Controle vondsten,['DB34'],1466,1466,0,0
4,DIAOPGRAVING,"['DC93', 'DB34', 'DC97']",882,882,0,0
5,AW1DeterminatiesJohan,['DB34'],802,802,0,0
6,SPOREN,"['DB34', 'DC97', 'DC112', 'DC93']",800,800,800,0
7,VULLINGEN,"['DC112', 'DC97', 'DB34', 'DC93']",679,679,0,0
8,ROMEINS AARDEWERK,['DB34'],594,594,0,594
9,VULLING,"['DC154', 'DC179']",526,526,0,0


## Detailanalyse Brondata naar Staging

In [9]:
df = df_brondata.merge(df_staging, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_brondata", "_staging"))
df[df.Stage_brondata.isnull() | df.Stage_staging.isnull() ]

,Table,Projecten_brondata,Aantal_Records,Aantal_tabellen,Stage_brondata,Projecten_staging,Stage_staging
36,AARDEWERK 1&2 VOORWERP,[DB34],96,1,Brondata,NaN,NaN
55,FOTO,NaN,88,2,NaN,"[DC179, DC154]",Staging


## Detailanalyse Staging naar SingleStore

In [10]:
df = df_staging.merge(df_singlestore, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_staging", "_singlestore"))
df[df.Stage_singlestore.isnull() | df.Stage_staging.isnull() ]

,Table,Projecten_staging,Aantal_Records,Aantal_tabellen,Stage_staging,Projecten_singlestore,Stage_singlestore
0,AARDEWERK 1,"[DB34, DC97]",4618,2,Staging,NaN,NaN
1,Aardewerk 1 backup,[DB34],4465,1,Staging,NaN,NaN
2,VONDSTENLIJST,"[DC11, DC112, DB34, DC97, DC93]",2121,5,Staging,NaN,NaN
3,Controle vondsten,[DB34],1466,1,Staging,NaN,NaN
4,DIAOPGRAVING,"[DC97, DC93, DB34]",882,3,Staging,NaN,NaN
5,AW1DeterminatiesJohan,[DB34],802,1,Staging,NaN,NaN
7,VULLINGEN,"[DC112, DC93, DC97, DB34]",679,4,Staging,NaN,NaN
8,ROMEINS AARDEWERK,[DB34],594,1,Staging,NaN,NaN
9,VULLING,"[DC179, DC154]",526,2,Staging,NaN,NaN
10,HOUT,"[DC11, DB34, DC36, DC93, DC112, DC97]",480,6,Staging,NaN,NaN


## Detailanalyse SingleStore naar Doelsysteem

In [11]:
df = df_singlestore.merge(df_doelsysteem, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_singlestore", "_doelsysteem"))
df[df.Stage_singlestore.isnull() | df.Stage_doelsysteem.isnull() ]

,Table,Projecten_singlestore,Aantal_Records,Aantal_tabellen,Stage_singlestore,Projecten_doelsysteem,Stage_doelsysteem
0,magazijnlijst,[MAGAZIJN],17887,1,SingleStore,NaN,NaN
1,doosnr,[MAGAZIJN],2732,1,SingleStore,NaN,NaN
3,SPOREN,"[DB34, DC97, DC93, DC112]",800,4,SingleStore,NaN,NaN
4,SPOOR,"[DC154, DC179]",479,2,SingleStore,NaN,NaN
5,GIS_SPOOR_STRUCTOBJ,[DC179],243,1,SingleStore,NaN,NaN
6,SPOOR_INHOUD,[DC179],208,1,SingleStore,NaN,NaN
7,REF_SPOORAARD,[DC112],96,1,SingleStore,NaN,NaN
10,magazijnlijst,NaN,5634,1,NaN,[MAGAZIJN],Doelsysteem
11,AARDEWERK 1,NaN,4618,2,NaN,"[DB34, DC97]",Doelsysteem
12,VONDSTENLIJST,NaN,2121,5,NaN,"[DB34, DC11, DC112, DC93, DC97]",Doelsysteem


In [12]:
# Experiments to be able to parse the mongo bson strings to either JSON or dict
# Not wirking :(((


startQ_regex = re.compile(r"({|\(|,\s*|:\s*)\'") # regex to replace first quote of words to double quaote
endQ_regex = re.compile(r"\'(:|\)|,|})") # regex to replace last quote of words to double quaote
regex = re.compile(r'ObjectId\((.*)\)') # regex to replace Object

# Does not work :(
def getFromJson(brondata):    
    if not brondata or brondata != "":
        brondata = brondata.replace('\"', '\'')
        brondata = startQ_regex.sub('\\1"', brondata)
        brondata = endQ_regex.sub('"\\1', brondata)
        brondata = regex.sub('\\1', brondata)
        brondata = brondata.replace('),', ',') # Quick fix because one ) could not be removed
        print(brondata)
        obj = json.loads(brondata)
        
        return obj
    else: 
        return {} 


#import re
#from bson.json_util import dumps, loads

#p = re.compile(r'blue (?P<animal>dog|cat)')
#p.sub(r'gray \g<animal>',s)
#re.sub(
#    pattern=r'ObjectId\((.*)\)', 
#    repl='\\1', 
#    string=str
#)

str = "{'_id': ObjectId('61e5f4a0ef919b0974d0395d'), 'LOKATIE': 'H:\\GEMEENTES\\PLAATS\\Opgravingen\\NAAM\\opgravingCODE.mdb', 'CODE': 'PN023', 'TOPONIEM': "Karitaat 'Molensloot", 'OPGRAVING': 'Vispaaiplaats Ruijven', 'CODENAAM': 'PN023', 'KAARTBLAD': '37E', 'XCOORD': 87523, 'YCOORD': 445002, 'JAAR': 2013, 'VONDSTENLIJST': 0, 'SPOREN': 0, 'DIAOPGRAVING': 0, 'DIAVOORWERP': 0, 'TEKENINGEN': 0, 'ROMEINS AARDEWERK': 0, 'AARDEWERK 1': 0, 'AARDEWERK 2': 0, 'KLEIPIJPEN': 0, 'TERRA COTTA': 0, 'GLAS': 0, 'BEEN': 0, 'BOT': 0, 'HOORN': 0, 'IVOOR': 0, 'HOUT': 0, 'METAAL': 0, 'MUNTEN EN PENNINGEN': 0, 'STEEN': 0, 'LEER': 0, 'TEXTIEL': 0, 'MODERN': 0, 'BIOLOGISCH': 0, 'SPECIAL': 0, 'IJZ': 0, 'ROM': 0, 'MIDDELEEUWS OF LATER': 0, 'VME': 0, 'LME': 0, 'LMEA': 0, 'LMEB': 0, 'NT': 0, 'WAARNEMING': 57944, 'table': 'OPGRAVINGEN', 'project': 'DELF-IT', 'bron': 'opgravingDELF-IT', 'loadtime': '2022-01-17T22:58:31+00:00'}"
#str = str.replace("\\", "/")
#str = str.replace("\'", "###")
#str = str.replace("\"", "\'")
#str = str.replace("###", "\"")


#str = re.sub(
#    pattern=r'ObjectId\((.*)\)', 
#    repl='\\1', 
#    string=str)

#json.loads(str)
#str = re.sub(
#    pattern=r"\'(.*?)\'(:|}|,)", 
#    repl='"\\1"\\2', 
#    string=str
#)
from ast import literal_eval
#d = eval(str)
#d['_id']

import yaml
yaml.load(str)

SyntaxError: invalid syntax (<ipython-input-12-fac76717d68a>, line 36)